## YOLO V4
- paper : (2020) YOLOv4 - Optimal Speed and Accuracy of Object Detection

- 낮은 FPS(실시간 X)와 너무나 큰 mini-batch-size로 인해 학습하는데 많은 수의 GPU들이 필요하다는 단점이 존재
- 일반적인 학습 환경에서도 높은 정확도와 빠른 object detector를 학습가능 (1 GPU만 있어도 충분)
- YOLO v4는 EfficientDet과 비슷한 AP 성능을 내면서, 2배 더 빠른 FPS를 보유. YOLO v3에 비해서 AP는 10%, FPS는 12% 향상

![yolov4](img/yolov4.png)

<hr>



### 주요 아이디어 목록
- 1) WRC (Weighted-Residual-Connections)
- 2) CSP (Cross-Stage-Partial-Connections)
- 3) CmBN (Cross mini-Batch Normalizations)
- 4) SAT (Self-Adversarial-Training)
- 5) Mish Activation
- 6) Mosaic Data Agumentation
- 7) Drop Block Regularization
- 8) CIOU Loss

## Detector 개념

### 1. Object Detection Models Structure
- 최신 detector는 주로 백본(Backbone)과 헤드(Head)라는 두 부분으로 구성 (추가적으로 넥(Neck))
    - **백본 : 입력 이미지를 feature map으로 변형**
    - **헤드 : Backbone에서 추출한 feature map의 location 작업을 수행**
        - predict classes
        - bounding boxes 작업이 수행
        
- Head 종류 2가지 (1stage vs 2stage)
    - **Sparse Prediction 헤드**
        - Two-Stage Detector는 대표적으로 Faster R-CNN, R-FCN
        - Predict Classes와 Bounding Box Regression 부분이 분리
    - **Dense Prediction 헤드**
        - One-Stage Detector는 대표적으로  YOLO, SSD
        - Predict Classes와 Bounding Box Regression이 통합
        
- **넥(Neck)은 Backbone과 Head를 연결하는 부분**
    - feature map을 refinement(정제), reconfiguration(재구성)함
    - ex) FPN(Feature Pyramid Network), PAN(Path Aggregation Network), BiFPN, NAS-FPN 등
    
### 2. Bag of Freebies (BOF)
- BOF는 **Inference cost의 변화 없이 (공짜로) 성능 향상이 되는 딥러닝 기법**
- ex) **Data Augmentation : CutMix, Mosaic 등** 
- ex) **Regularization : Dropout, Dropblock, DropPath 등**
- ex) **BBox Regression의 loss 함수 : IOU loss, CIOU loss 등 **

![yolov4_bag_of_freebies](img/yolov4_bag_of_freebies.png)


### 3. Bag of Specials (BOS)
- BOS는 BOF의 반대로, **Inference cost가 조금 상승하지만, 성능 향상이 되는 딥러닝 기법**
- **ex) Enhance Receptive filed : SPP, ASPP, RFB **
- **ex) Feature Integration : Skip-connection, Hyper-column, Bi-FPN **
- **ex) 최적 Activation function : P-ReLU, ReLU6, Mish**

    ![yolov4_bag_of_specials](img/yolov4_bag_of_specials.png)

![yolov4_selection_bof_bos](img/yolov4_selection_bof_bos.png)


<hr>

## YOLO V4 Architecture
- **1) Backbone : CSP-Darkent53**
- **2) Neck : SPP(Spatial Pyramid Pooling), PAN(Path Aggregation Network)**
- **3) Head : YOLO-v3**
- **4) BoF & BoS 기법들**

![yolov4_selection_bof_bos](img/yolov4_selection_bof_bos.png)

### BOF
- 데이터 증강법(Data Augmentation)
    - **CutMix : 한 이미지에 2개의 class를 넣은 것**
    - **Mosaic Data Augmentation : 한 이미지에 4개의 class를 넣은 것**
    
    ![yolov4_mixup_cutmix](img/yolov4_mixup_cutmix.png)
    
- Regularization (규제) 
    - **DropBlock Regularization : 랜덤하게 out시키는 DropOut과는 달리, 일정한 범위를 out 시킴**

- Smooth Labeling
    - label을 0.1, 0.9와 같은 확률로 표시 / label을 0 또는 1이 아니라 smooth하게 부여하여, calibration 및 regularization 효과를 얻게 되어 overfitting 문제를 방지

### BOS (Bag of Specials)
- Activation
    - **Mish Activation : 활성화 함수에 관한 최신 기법**
        - f(x) = x * tanh(softplus(x)) / softplus(x) = ln(1 + e^x)
        - 0에 가까운 기울기로 인해, 훈련 속도가 급격히 느려지는 포화(Saturation) 문제 방지
            - ReLu보다 Mish의 gradient가 좀 더 smooth함
        - 강한 규제(Regularation) 효과로 overfitting 문제 방지
    - 학습이 어려워서 PReLU, SELU를 제외하였고, ReLU6도 quantized network에 특화된 함수여서 제외
    
![yolov4_mish_activation](img/yolov4_mish_activation.png)

- Enhance Receptive Field
    - **CSP(Cross-Stage Partial connections) : 기존의 CNN 네트워크의 연산량을 줄이는 기법**
        - 원문에서 제안하는 CSP Net은 연산량을 20% 줄이면서 높은 AP를 가짐
        - CSPNet은 굉장히 heavy한 inference cost를 완화시키며 정확도 손실을 최소로 할 수 있는 Cross Stage Partial Network 구조를 제안
        - input feature map을 2개의 part로 나눈 뒤, 하나의 part는 연산에 참여시키지 않고 뒤에서 합쳐주는 방식을 기반으로 inference cost, memory cost 등을 줄임
        - 학습 관점에서는 gradient flow를 나눠줘서 학습에 좋은 영향을 줘서 정확도 손실이 적다고 주장
        - CSPNet은 input feature net을 2파트로 나누어 절반만 연산의 포함시키며 연산 후에 concat으로 합침
        - CSPResNext50 / CSPDarknet53 / EfficientNet-B3 중 CSPDarkNet53 선택 (처리속도가 제일빠름)

![yolov4_cspnet](img/yolov4_cspnet.png)

<hr>



### 새로운 기술 제안
- Mosiac augmentation
    - 네가지 이미지를 하나의 이미지로 합쳐줌
    - 한개의 이미지로 네개의 이미지 학습 효과 (=배치 사이즈가 4배)
    
![yolov4_mosiac](img/yolov4_mosiac.png)


#### Self-Adversarial Training (SAT)
- Adversarial attack을 가한 이미지를 기존의 gt(ground-truth)로 학습하여, 모델이 좀더 디테일한 영역을 학습 시키게 함
    
![yolov4_self_advertising](img/yolov4_self_advertising.png)

#### CmBN (Cross mini-Batch Normalizations)
- Batch Norm을 수정한 CmBN을 제안

![yolov4_cmbn](img/yolov4_cmbn.png)

#### Modified SAM and PAN
![yolov4_sam_pan](img/yolov4_sam_pan.png)

<hr>

## YOLO V4 Experimental Result

### Classification Result

- CutMix, Mosaic Augmentation과 Label Smoothing을 적용하여 성능이 약간 향상
- Activation Function으로는 ReLU대비 Swish는 오히려 안 좋은 결과를 보임
- Mish를 사용할 때 큰 폭의 성능 향상

![yolov4_classification_result](img/yolov4_classification_result.png)


### Object Detection Result
- M : Mosaic Augmentation을 의미
- GA :  Mosaic를 제안한 Glenn Jocher의 도움을 받아서 진행한 방식, 유전 알고리즘을 이용한 hyper parameter search를 의미
- CBN : Cross mini-Batch Normalization
- CA : Cosine Annealing Learning Rate scheduling을 의미
- OA : YOLOv2에서 제안된 k-means 기반의 사전에 정해 놓은 anchor 크기를 사용하는 것을 의미
- 512x512 input resolution에 최적화된 anchor들을 사용
- loss : MSE, GIoU, DIoU, CIoU 등을 사용
- 각각을 적용하였을 때의 Ablation study 결과 제시

![yolov4_result](img/yolov4_result.png)

<hr>

## 참고문헌
- https://ropiens.tistory.com/33
- https://hoya012.github.io/blog/yolov4/